In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet_preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Constants
IMAGE_SIZE = [224, 224]
BATCH_SIZE = 32
NUM_EPOCHS = 15

In [4]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Dataset paths
train_path = '/content/drive/MyDrive/Underwater/dataset/Train'
validation_path = '/content/drive/MyDrive/Underwater/dataset/Validation'


In [6]:
# Load ResNet50 model with pre-trained weights
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


94765736/94765736 [==============================] - 0s 0us/step


In [7]:
# Freeze the layers of the pre-trained model
for layer in resnet.layers:
    layer.trainable = False

In [9]:
# Number of classes
folders_resnet = glob('/content/drive/MyDrive/Underwater/dataset/Train/*')
num_classes_resnet = len(folders_resnet)
num_classes_resnet

3

In [10]:
# Create a custom neural network model using a pre-trained ResNet50 base
x_resnet = Flatten()(resnet.output)
prediction_resnet = Dense(num_classes_resnet, activation='softmax')(x_resnet)
model_resnet = Model(inputs=resnet.input, outputs=prediction_resnet)

In [11]:
# Display a summary of the ResNet50-based model architecture
model_resnet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [12]:
# Compile the ResNet50-based model
adam = optimizers.Adam()
model_resnet.compile(loss='categorical_crossentropy',
                     optimizer=adam,
                     metrics=['accuracy'])

In [13]:
# Training Augmentation for ResNet50
train_datagen_resnet = ImageDataGenerator(
    preprocessing_function=resnet_preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [14]:
validation_datagen_resnet = ImageDataGenerator(
    preprocessing_function=resnet_preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [15]:
# Create data generators for ResNet50
train_set_resnet = train_datagen_resnet.flow_from_directory(train_path,
                                                             target_size=(224, 224),
                                                             batch_size=BATCH_SIZE,
                                                             class_mode='categorical')

validation_set_resnet = validation_datagen_resnet.flow_from_directory(validation_path,
                                                                      target_size=(224, 224),
                                                                      batch_size=BATCH_SIZE,
                                                                      class_mode='categorical')

Found 2594 images belonging to 3 classes.
Found 583 images belonging to 3 classes.


In [16]:
# Set up callbacks and start training
checkpoint_resnet = ModelCheckpoint(filepath='mymodel_resnet.h5',
                                    verbose=2, save_best_only=True)
callbacks_resnet = [checkpoint_resnet]

start_resnet = datetime.now()

In [17]:
# Train the ResNet50-based model
model_resnet_history = model_resnet.fit_generator(
    train_set_resnet,
    validation_data=validation_set_resnet,
    epochs=NUM_EPOCHS,
    steps_per_epoch=len(train_set_resnet),
    validation_steps=len(validation_set_resnet),
    callbacks=callbacks_resnet, verbose=2
)

duration_resnet = datetime.now() - start_resnet
print("Training completed in time: ", duration_resnet)

<ipython-input-17-c49d57ba1008>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_resnet_history = model_resnet.fit_generator(


Epoch 1/15

Epoch 1: val_loss improved from inf to 0.48732, saving model to mymodel_resnet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


82/82 - 620s - loss: 1.9147 - accuracy: 0.9237 - val_loss: 0.4873 - val_accuracy: 0.9640 - 620s/epoch - 8s/step
Epoch 2/15

Epoch 2: val_loss did not improve from 0.48732
82/82 - 54s - loss: 0.3627 - accuracy: 0.9773 - val_loss: 0.7149 - val_accuracy: 0.9640 - 54s/epoch - 664ms/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.48732
82/82 - 55s - loss: 0.3914 - accuracy: 0.9796 - val_loss: 0.5952 - val_accuracy: 0.9674 - 55s/epoch - 671ms/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.48732
82/82 - 55s - loss: 0.4268 - accuracy: 0.9807 - val_loss: 0.5827 - val_accuracy: 0.9708 - 55s/epoch - 670ms/step
Epoch 5/15

Epoch 5: val_loss did not improve from 0.48732
82/82 - 54s - loss: 0.2242 - accuracy: 0.9854 - val_loss: 0.5084 - val_accuracy: 0.9777 - 54s/epoch - 656ms/step
Epoch 6/15

Epoch 6: val_loss did not improve from 0.48732
82/82 - 57s - loss: 0.1899 - accuracy: 0.9892 - val_loss: 1.4458 - val_accuracy: 0.9657 - 57s/epoch - 700ms/step
Epoch 7/15

Epoch 7: val_los